In [111]:
import json
from enum import Enum
from gurobipy import Model, GRB, quicksum
import numpy as np



In [112]:
class DATASET(Enum):
    TOY = "toy"
    MEDIUM = "medium"
    LARGE = "large"

In [113]:
def load_data(name):
    """name must be an instance of DATASET like DATASET.TOY for example"""
    if not isinstance(name, DATASET):
        raise TypeError("direction must be an instance of DATASET Enum")
    with open(f"../data/{name.value}_instance.json", "r") as f:
        data = json.load(f)
    return data


def get_dims(data):
    return (
        len(data["staff"]),
        data["horizon"],
        len(data["qualifications"]),
        len(data["jobs"]),
    )

def get_qualification_index(list_qualifications, qualification): # qualification is "A", "B", "C" ...
    return list_qualifications.index(qualification)

In [114]:
def init_model():
    m = Model("Project modelling")
    return m

In [115]:
def create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs):
    X = model.addVars(n_staff, horizon, n_qualifs, n_jobs, vtype=GRB.BINARY, name="assignements")
    J = model.addVars(n_jobs, vtype=GRB.BINARY, name="completion")
    D = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="end_dates")
    L = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="nb_dates_late")
    return model, X, J, D, L


In [116]:
def add_constraints_end_dates(model, X, D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        X[i, j, k, l] * j <= D[l] 
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [117]:
def add_constraints_lateness(model, D, L, jobs, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        D[l] - jobs[l]["due_date"] <= L[l] for l in range(n_jobs)
    )

In [118]:
def add_constraints_worked_days_below_required_days(model, X, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) <= jobs[l]["working_days_per_qualification"][k] for l in range(n_jobs) for k in jobs[l]["working_days_per_qualification"].keys())
    

In [125]:
def add_constraints_worked_days_above_required_days(model, X ,J,data,n_staff,n_jobs,horizon):
    for l in range(n_jobs): 
        for quali,njk in data["jobs"][l]["working_days_per_qualification"].items():
            model.addConstr(J[l]*njk <= quicksum([X[i,j,get_qualification_index(data["qualifications"], quali),l] for i in range(n_staff) for j in range(horizon)]))

In [126]:
def in_qualification(i, k, data):
    data = {l : data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["qualifications"]
    return k in data


def add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if not in_qualification(i, k, data)
    )


def in_vacation(i, j, data):
    data = {l: data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["vacations"]
    return j in data


def add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if in_vacation(i, j, data)
    )

In [127]:
def add_constraint_employees_working_only_one_day(model, X ,J,data,n_staff,n_jobs,horizon, n_qualifs):
    model.addConstrs(quicksum(X[i,j,k,l] for l in range(n_jobs) for k in range (n_qualifs)) <= 1 for i in range(n_staff) for j in range(horizon))

In [131]:
def add_profit_as_first_objective(model, J, L, jobs):

    benef = sum(
        [
            J[index_job] * (job["gain"] - job["daily_penalty"] * L[index_job])
            for index_job, job in enumerate(jobs)
        ]
    )

    model.setObjective(benef, GRB.MAXIMIZE)

In [132]:
def main():
    # Importing data
    data = load_data(DATASET.TOY)
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)

    # Instanciation du modèle
    model = init_model()

    # Création des variables : binaires dans X et J, entières de 0 à horizon + 3
    model, X, J, D, L = create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs)

    # maj du modèle
    model.update()

    # Ajout des constraintes

    add_constraint_employees_working_only_one_day(model, X ,J,data,n_staff,n_jobs,horizon, n_qualifs)
    add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)

    add_constraints_end_dates(model, X, D, n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_lateness(model, D, L, data["jobs"], n_staff, horizon, n_qualifs, n_jobs)

    add_constraints_worked_days_below_required_days(model, X, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
    add_constraints_worked_days_above_required_days(model, X ,J,data,n_staff,n_jobs,horizon)

    # Fonction Objectif
    add_profit_as_first_objective(model, J, L, data["jobs"])

    # Paramétrage (mode mute)
    # model.params.outputflag = 0

    # Résolution du PL
    model.optimize()

In [133]:
main()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 520 rows, 240 columns and 1205 nonzeros
Model fingerprint: 0xc0e66a65
Model has 5 quadratic objective terms
Variable types: 0 continuous, 240 integer (230 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+01, 2e+01]
  QObjective range [6e+00, 6e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 520 rows and 240 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000

In [ ]:
# # Objective Function 1: Maximize Profits
# model.setObjective(sum(profit[j]*y[j] for j in projects) - sum(penalty[i,j]*x[i,j,k] for i in staff for j in qualifications for k in working_days), GRB.MAXIMIZE)

# # Objective Function 2: Minimize Number of Projects per Employee
# model.setObjective(sum(x[i,j,k] for i in staff for j in qualifications for k in working_days) , GRB.MINIMIZE)
# model.setObjective(sum(x[i,j,k] for i in staff for j in qualifications for k in working_days) , GRB.MINIMIZE)

# # Objective Function 3: Minimize Length of Longest Project
# model.setObjective(max(duration[j] for j in projects), GRB.MINIMIZE)